## Day 12
https://adventofcode.com/2025/day/12

In [1]:
import numpy as np
import time

In [2]:
def read_input_12(filename):
    with open(filename) as f:
        situation = f.read().split("\n\n")
        shapes = [np.array([[1 if c=="#" else 0 for c in r] for r in b.split("\n")[1:] ]) for b in situation[:-1]]
        regions = []
        for r in situation[-1].strip("\n").split("\n"):
            t = r.split(": ")
            size = tuple([int(i) for i in t[0].split("x")])
            tiles = [int(i) for i in t[1].split(" ")]
            regions.append((size, tiles))
    return shapes, regions

In [3]:
def all_rotations_and_flips(mat):
    """Return unique transformations of a 2D matrix."""
    transforms = []
    
    # 4 rotations
    for k in range(4):
        transforms.append(np.rot90(mat, k))
    
    # Flipped versions
    flipped = np.flip(mat, axis=1)
    for k in range(4):
        transforms.append(np.rot90(flipped, k))
    
    # Remove duplicates by converting to tuples
    unique = []
    seen = set()
    for t in transforms:
        key = t.tobytes()
        if key not in seen:
            seen.add(key)
            unique.append(t)
    
    return unique

def precompute_placements(board_shape, shape_variants):
    """
    Precompute all valid (r, c) positions where each shape variant can be placed.
    Returns list of (variant_idx, row, col, positions_array) tuples.
    positions_array is a list of (row, col) coordinates where the shape has a 1.
    """
    nr, nc = board_shape
    placements = []
    for v_idx, variant in enumerate(shape_variants):
        vr, vc = variant.shape
        # Get positions of 1 in the variant
        positions = [(r, c) for r in range(vr) for c in range(vc) if variant[r, c] == 1]
        for r in range(nr - vr + 1):
            for c in range(nc - vc + 1):
                # Translate positions to board coordinates
                board_positions = [(r + pr, c + pc) for pr, pc in positions]
                placements.append((v_idx, r, c, board_positions)) 
    return placements

def can_place(board, positions):
    """Check if all positions are free on the board."""
    return all(board[r, c] == 0 for r, c in positions)

def place(board, positions, value=1):
    """Place a shape on the board."""
    for r, c in positions:
        board[r, c] = value

def unplace(board, positions):
    """Remove a shape from the board."""
    for r, c in positions:
        board[r, c] = 0

def backtrack(board, shape_placements, remaining_shapes):
    """
    Backtracking with in-place board modifications
    - board: numpy array representing the current board state
    - shape_placements: dict mapping shape_idx -> list of precomputed placements
    - remaining_shapes: list of shape indices still to place
    """
    if not remaining_shapes:
        return True
    
    # Pick next shape to place
    shape_idx = remaining_shapes[0]
    placements = shape_placements[shape_idx]
    
    # Try each placement
    for v_idx, r, c, positions in placements:
        if can_place(board, positions):
            # Place the shape
            place(board, positions)
            
            # Recurse
            if backtrack(board, shape_placements, remaining_shapes[1:]):
                return True
            
            # Backtrack: remove the shape
            unplace(board, positions)
    
    return False

def solve_region(region, shapes):
    """Solve a region using backtracking"""
    size, tiles = region
    nr, nc = size
    
    # Build list of shape indices to place
    shape_list = []
    for i, count in enumerate(tiles):
        shape_list.extend([i] * count)
    
    # Sort shapes by size (larger first) by counting filled cells for each shape
    # It should helps to prune search space during backtracking (hopeless solutions are reached earlier)
    shape_sizes = [(i, np.sum(shapes[i])) for i in range(len(shapes))]
    shape_list.sort(key=lambda idx: shape_sizes[idx][1], reverse=True)
    
    # Precompute all transformations for each unique shape
    shape_variants = {}
    for shape_idx in set(shape_list):
        shape_variants[shape_idx] = all_rotations_and_flips(shapes[shape_idx])
    
    # Precompute all valid placements for each shape
    shape_placements = {}
    for shape_idx in set(shape_list):
        shape_placements[shape_idx] = precompute_placements((nr, nc), shape_variants[shape_idx])
    
    # Initialize board
    board = np.zeros((nr, nc), dtype=np.int8)
    
    return backtrack(board, shape_placements, shape_list)

def part1_noopt(filename, verbose=False):
    
    start_time = time.time()
    shapes, regions = read_input_12(filename)
    count = 0
    
    for i, region in enumerate(regions):
        region_start = time.time()
        solved = solve_region(region, shapes)
        region_time = time.time() - region_start
        
        if verbose: 
            print(f"Region {i+1}: {region[0]} with tiles {region[1]} -> {solved} (took {region_time:.3f}s)")
        if solved:
            count += 1
    
    if verbose: 
        print(f"\nRegions that can fit selected shapes: {count}")

    total_time = time.time() - start_time        
    print(f"Total execution time: {total_time:.3f}s")
    
    return count

In [4]:
part1_noopt("examples/example12.txt",verbose=True)

Region 1: (4, 4) with tiles [0, 0, 0, 0, 2, 0] -> True (took 0.000s)
Region 2: (12, 5) with tiles [1, 0, 1, 0, 2, 2] -> True (took 0.002s)
Region 3: (12, 5) with tiles [1, 0, 1, 0, 3, 2] -> False (took 99.186s)

Regions that can fit selected shapes: 2
Total execution time: 99.191s


2

If a solution exists, it is found rather quickly. On the other hand, **if a solution does not exist, backtracking takes a long time to explore the full phase space (and there are 1000 large regions with lots of shapes to be tested in the full input!).** The correct solution to this would be to implement pruning strategies, but before attempting it **I will try to simply top the solving procedure that takes too long to converge ;-)**. This is obviously dangerous, since it does not guarantee that a solution could ultimately be found in longer searches...

In [5]:
START_TIME = None
TIME_LIMIT = 2.0 # seconds

def timed_backtrack(board, shape_placements, remaining_shapes):
    if time.time() - START_TIME > TIME_LIMIT:
        return False # abort early

    if not remaining_shapes:
        return True
    
    shape_idx = remaining_shapes[0]
    placements = shape_placements[shape_idx]
    
    for v_idx, r, c, positions in placements:
        if can_place(board, positions):
            place(board, positions)
            if timed_backtrack(board, shape_placements, remaining_shapes[1:]):
                return True
            unplace(board, positions)
    
    return False

def solve_region_timed(region, shapes, time_limit=2.0):
    global START_TIME, TIME_LIMIT
    TIME_LIMIT = time_limit
    START_TIME = time.time()
    
    size, tiles = region
    nr, nc = size
    
    shape_list = []
    for i, count in enumerate(tiles):
        shape_list.extend([i] * count)
    
    shape_sizes = [(i, np.sum(shapes[i])) for i in range(len(shapes))]
    shape_list.sort(key=lambda idx: shape_sizes[idx][1], reverse=True)
    
    shape_variants = {}
    for shape_idx in set(shape_list):
        shape_variants[shape_idx] = all_rotations_and_flips(shapes[shape_idx])
    
    shape_placements = {}
    for shape_idx in set(shape_list):
        shape_placements[shape_idx] = precompute_placements((nr, nc), shape_variants[shape_idx])
    
    board = np.zeros((nr, nc), dtype=np.int8)
    return timed_backtrack(board, shape_placements, shape_list)

def part1_timed(filename, time_limit=2, verbose=False):
    
    start_time = time.time()
    shapes, regions = read_input_12(filename)
    count = 0
    
    for i, region in enumerate(regions):
        region_start = time.time()
        solved = solve_region_timed(region, shapes, time_limit)
        region_time = time.time() - region_start
        
        if verbose: 
            print(f"Region {i+1}: {region[0]} with tiles {region[1]} -> {solved} (took {region_time:.3f}s)")
        if solved:
            count += 1
    
    if verbose: 
        print(f"\nRegions that can fit selected shapes: {count}")

    total_time = time.time() - start_time        
    print(f"Total execution time: {total_time:.3f}s")
    
    return count

In [6]:
part1_timed("examples/example12.txt", time_limit=2, verbose=True)

Region 1: (4, 4) with tiles [0, 0, 0, 0, 2, 0] -> True (took 0.000s)
Region 2: (12, 5) with tiles [1, 0, 1, 0, 2, 2] -> True (took 0.001s)
Region 3: (12, 5) with tiles [1, 0, 1, 0, 3, 2] -> False (took 2.000s)

Regions that can fit selected shapes: 2
Total execution time: 2.002s


2

In [8]:
part1_timed("AOC2025inputs/input12.txt", time_limit=2, verbose=True)

Region 1: (42, 47) with tiles [40, 32, 40, 31, 35, 31] -> True (took 0.212s)
Region 2: (49, 43) with tiles [45, 42, 24, 36, 38, 39] -> True (took 0.154s)
Region 3: (44, 49) with tiles [57, 61, 54, 55, 54, 49] -> False (took 3.914s)
Region 4: (42, 37) with tiles [37, 46, 34, 36, 42, 45] -> False (took 3.001s)
Region 5: (36, 38) with tiles [30, 21, 24, 26, 24, 19] -> True (took 0.068s)
Region 6: (35, 46) with tiles [53, 39, 44, 37, 38, 36] -> False (took 2.941s)
Region 7: (39, 50) with tiles [37, 37, 32, 22, 38, 42] -> True (took 0.121s)
Region 8: (48, 46) with tiles [48, 42, 28, 43, 30, 48] -> True (took 0.150s)
Region 9: (45, 42) with tiles [29, 34, 30, 46, 32, 38] -> True (took 0.100s)
Region 10: (47, 47) with tiles [50, 62, 57, 64, 54, 52] -> False (took 4.088s)
Region 11: (49, 37) with tiles [51, 43, 47, 38, 49, 53] -> False (took 3.304s)
Region 12: (40, 37) with tiles [23, 18, 33, 21, 39, 22] -> True (took 0.074s)
Region 13: (43, 44) with tiles [45, 32, 32, 36, 22, 29] -> True (too

575

**... and the dirty approach works!!! ;-)**